# NavEph2PosECEF Walkthrough

This notebook walks through the most important pieces of `NavEph2PosECEF.py`, the script that parses RINEX navigation files and turns them into satellite positions. The focus is on:

1. `parse_rinex_nav_v2` – turning the plain-text RINEX navigation file into structured ephemeris dictionaries.
2. `compute_positions_for_day` – consuming those ephemerides to produce ECEF coordinates.


In [28]:
PRNS = (1,2,3) # PRN selection

In [29]:
import numpy as np

from NavEph2PosECEF import (  # import the implementations to reference
    C, MU, OMEGA_E, # Physical constants
    parse_rinex_nav_v2,  # parser function
    compute_positions_for_day,  # propagator function
    TRACK_COLORS,  # plotting color palette
    calendar2mjd,  # calendar to MJD converter
    mjd_from_doy_year, # Year/DoY to MJD converter
    nsteffensen,  # Kepler solver
    _rinex_numbers,  # numeric tokenizer helper
)  # end import list

print('Loaded functions from NavEph2PosECEF.py')  # quick import confirmation


Loaded functions from NavEph2PosECEF.py


## Function: `parse_rinex_nav_v2`

Below is the function copied from the script with detailed commentary. These comments emphasize the parsing logic for each block of lines in the RINEX navigation file.

In [30]:
def parse_rinex_nav_v2(path): 
    """Annotated replica of parse_rinex_nav_v2."""
    # Create an empty list to hold all ephemeris dictionaries read from the file.
    ephs = []  # storage for parsed ephemerides

    # Open the navigation file using ASCII encoding and ignore any decoding errors.
    with open(path, 'r', encoding='ascii', errors='ignore') as f:  # safe text reader

        # Consume lines until the "END OF HEADER" marker that finishes the RINEX header.
        for line in f:  # iterate through header lines
            if 'END OF HEADER' in line:  # stop once marker is found
                break  # exit the header-reading loop

        # Read ephemeris blocks until the file ends.
        while True:  # unlimited loop over ephemeris blocks
            # Read the first line of the next ephemeris block.
            head = f.readline()  # fetch block header
            
            # Stop looping when EOF is reached.
            if not head:  # EOF check
                break  # leave main loop when file ends
            
            # Normalize scientific notation by replacing FORTRAN style D/d with E/e.
            head = head.replace('D','E').replace('d','e')  # standardize exponent letters
            
            # Skip any stray blank lines before the next header.
            if not head.strip():  # ignore empty lines
                continue  # fetch the next potential header
            
            # Split the header.
            toks = head.split()  # tokenized header line
            if len(toks) >= 10:  # branch for compact spacing format
                # Newer format: PRN is in column 0 and the timestamp follows as integers.
                prn, yy, mm, dd, hh, mn = int(toks[0]), int(toks[1]), int(toks[2]), int(toks[3]), int(toks[4]), int(toks[5])  # parse identifiers
                # Fractional seconds and clock bias/drift/drift-rate close the header line.
                ss, af0, af1, af2 = float(toks[6]), float(toks[7]), float(toks[8]), float(toks[9])  # parse clock terms
            else:  # branch for legacy fixed-width layout
                # Legacy fixed-width layout: PRN occupies the first two characters.
                prn = int(head[0:2])  # two-character PRN code
                # Each subsequent field spans two or more character columns.
                yy = int(head[3:5]); mm = int(head[6:8]); dd = int(head[9:11])  # extract calendar parts
                hh = int(head[12:14]); mn = int(head[15:17]); ss = float(head[18:22])  # extract clock time
                # Satellite clock parameters occupy fixed-width slices later on the line.
                af0 = float(head[22:41]); af1 = float(head[41:60]); af2 = float(head[60:79])  # fixed-width clock terms

            # Collect the six data lines that follow the header.
            L = []  # container for the six lines
            for _ in range(6):  # iterate fixed number of lines
                s = f.readline()  # read line
                if not s: s = ""  # guard against truncated file
                L.append(s.replace('D','E').replace('d','e'))  # normalize exponents and store
            # The seventh line contains TOT/FIT information; keep a normalized copy as well.
            tail = f.readline()  # read TOT/FIT line
            if not tail: tail = ""  # handle missing line
            tail = tail.replace('D','E').replace('d','e')  # normalize TOT/FIT line
            # Split each data line using the helper that can break glued scientific values.
            t1 = _rinex_numbers(L[0]); t2 = _rinex_numbers(L[1]); t3 = _rinex_numbers(L[2])  # first half of values
            t4 = _rinex_numbers(L[3]); t5 = _rinex_numbers(L[4]); t6 = _rinex_numbers(L[5])  # second half of values
            t7 = _rinex_numbers(tail)  # TOT and FIT fields

            # Build the ephemeris dictionary assigning each field to a descriptive key.
            eph = dict(  # assemble dictionary with meaningful keys
                prn = prn,  # satellite PRN identifier
                tocy = yy, tocm = mm, tocd = dd, toch = hh, tocn = mn, tocs = float(ss),  # time of clock components
                af0 = float(af0), af1 = float(af1), af2 = float(af2),  # clock bias, drift, drift rate
                iode = int(float(t1[0])), crs = float(t1[1]), dn = float(t1[2]), M0 = float(t1[3]),  # first data line values
                cuc = float(t2[0]), e = float(t2[1]), cus = float(t2[2]), sqrtA = float(t2[3]),  # second data line values
                toe = float(t3[0]), cic = float(t3[1]), Omega0 = float(t3[2]), cis = float(t3[3]),  # third data line values
                i0 = float(t4[0]), crc = float(t4[1]), omega = float(t4[2]), Omegadot = float(t4[3]),  # fourth data line values
                idot = float(t5[0]), gpsweek = int(float(t5[2])),  # fifth data line: inclination rate and GPS week
                svacc = float(t6[0]) if len(t6)>0 else 0.0,  # satellite accuracy flag
                health = float(t6[1]) if len(t6)>1 else 0.0,  # satellite health status
                tgd = float(t6[2]) if len(t6)>2 else 0.0,  # group delay
                iodc = int(float(t6[3])) if len(t6)>3 else int(float(t1[0])),  # issue of data clock fallback
                tot = float(t7[0]) if len(t7)>0 else float(t3[0]),  # time of transmission seconds
                fit = float(t7[1]) if len(t7)>1 else 0.0  # fit interval hours
            )  # end of dictionary construction
            
            # Derived fields: square the semi-major axis root to get the actual A.
            eph['A'] = eph['sqrtA'] * eph['sqrtA']  # semi-major axis in meters
            # Expand two-digit year into civil year using convention (>= 51 => 1900s).
            tocy_full = 1900 + yy if yy > 50 else 2000 + yy  # resolve two-digit year
            # Convert the Time Of Clock entries into Modified Julian Date for comparisons.
            eph['tocmjd'] = calendar2mjd(tocy_full, mm, dd, hh, mn, float(ss))  # TOC as MJD
            # Convert TOE and TOT from GPS week + seconds of week into Modified Julian Date.
            eph['toemjd'] = 44244 + eph['gpsweek'] * 7.0 + eph['toe'] / 86400.0  # TOE in MJD
            eph['totmjd'] = 44244 + eph['gpsweek'] * 7.0 + eph['tot'] / 86400.0  # TOT in MJD
            # Add the populated dictionary to the result list.
            ephs.append(eph)  # store structured ephemeris
    
    # Return the entire collection of parsed ephemeris entries.
    return ephs  # deliver list of dictionaries

## Function: `compute_positions_for_day`

This routine iterates across seconds-of-day and satellites to locate the best ephemeris, compute the orbital parameters, and emit ECEF positions. The heavily commented version below explains each step.

In [31]:
def compute_positions_for_day(per_sat, doy, yy, step=5):  # orbital propagator

    # Build a lookup of Time Of Ephemeris values (in MJD) for each PRN to speed searches.
    sat_toe = { prn: np.array([e['toemjd'] for e in per_sat[prn]], dtype=float)  # map PRN to TOE array
                for prn in PRNS if per_sat.get(prn) }  # precompute TOE arrays per PRN

    # Iterate over the seconds of day using the requested step size (default 5 seconds).
    # ------------------------------------------------------------------------------------------
    for sod in range(0, 86400, step):  # walk through the day
        # Convert the current day-of-year, year, and second-of-day into Modified Julian Date.
        mjd = mjd_from_doy_year(doy, yy, sod)  # observation time as MJD
        
        # Loop over PRNs in the canonical order.
        # ------------------------------------------------------------------------------------------
        for prn in PRNS:  # examine each satellite
            # Skip satellites that have no ephemeris entries.
            if prn not in sat_toe:  # no data for PRN
                continue  # skip to next PRN
            # Fetch the list of ephemeris dictionaries for this satellite.
            L = per_sat[prn]  # ephemeris list for PRN
            # Retrieve the precomputed array of toe values to find the closest match quickly.
            toes = sat_toe[prn]  # numpy array of toe values
            # Select the ephemeris whose toe is closest to the target time.
            idx = int(np.argmin(np.abs(toes - mjd)))  # index of nearest ephemeris
            e = L[idx]  # chosen ephemeris dictionary
            # Compute time difference between request and toe in seconds.
            tk = (mjd - e['toemjd']) * 86400.0  # seconds since ephemeris reference
            # Skip ephemerides older than 2 hours to mimic ICD recommendations.
            if abs(tk) > 7200.0:  # enforce allowable age limit
                continue  # skip outdated ephemeris
            # Compute delta-time from the clock reference time to evaluate the clock polynomial.
            dt = (mjd - e['tocmjd']) * 86400.0  # seconds since clock reference time
            
            # Evaluate the clock correction (converted to meters using speed of light).
            b = (e['af0'] + e['af1'] * dt + e['af2'] * dt * dt) * C  # clock bias in meters
            
            # Retrieve the semi-major axis squared value.
            a = e['A']  # semi-major axis in meters
            # Compute the mean motion from Kepler's third law for the nominal orbit.
            n0 = np.sqrt(MU / (a ** 3))  # computed mean motion
            # Apply the delta-n correction to get the corrected mean motion.
            n = n0 + e['dn']  # corrected mean motion
            # Calculate the mean anomaly at the current time.
            Mk = e['M0'] + n * tk  # mean anomaly
            # Solve Kepler's equation for the eccentric anomaly using the Newton-Steffensen method.
            Ek = nsteffensen(Mk, e['e'])  # eccentric anomaly solution
            # Precompute sine and cosine of the eccentric anomaly for reuse.
            sinE = np.sin(Ek); cosE = np.cos(Ek)  # sine and cosine of eccentric anomaly
            # Compute the true anomaly using the arctangent formulation.
            nu = np.arctan2(np.sqrt(1.0 - e['e'] * e['e']) * sinE, (cosE - e['e']))  # true anomaly
            # The argument of latitude is the true anomaly plus the argument of perigee.
            phi = nu + e['omega']  # argument of latitude
            # Doubled argument used in the harmonic correction terms.
            two_phi = 2.0 * phi  # auxiliary double-angle
            # Latitude correction from sine/cosine harmonics.
            duk = e['cus'] * np.sin(two_phi) + e['cuc'] * np.cos(two_phi)  # delta argument of latitude
            # Radius correction from sine/cosine harmonics.
            drk = e['crs'] * np.sin(two_phi) + e['crc'] * np.cos(two_phi)  # delta radius
            # Inclination correction from sine/cosine harmonics.
            dik = e['cis'] * np.sin(two_phi) + e['cic'] * np.cos(two_phi)  # delta inclination
            # Apply the latitude correction to get the corrected argument of latitude.
            uk = phi + duk  # corrected argument of latitude
            # Apply the radius correction and compute the current orbital radius.
            rk = a * (1.0 - e['e'] * cosE) + drk  # corrected orbital radius
            # Apply the inclination correction plus the time-dependent idot term.
            ik = e['i0'] + dik + e['idot'] * tk  # corrected inclination
            # Position in the orbital plane prior to Earth rotation correction.
            x_orb = rk * np.cos(uk)  # orbital plane X
            y_orb = rk * np.sin(uk)  # orbital plane Y
            # Compute the longitude of ascending node corrected for Earth rotation.
            Omgk = e['Omega0'] + (e['Omegadot'] - OMEGA_E) * tk - OMEGA_E * e['toe']  # corrected node longitude
            # Precompute the sine and cosine of the corrected node angle.
            cosO = np.cos(Omgk); sinO = np.sin(Omgk)  # trig of node angle
            # Precompute sine and cosine of the inclination for reuse.
            cosi = np.cos(ik);   sini = np.sin(ik)  # trig of inclination
            # Rotate from orbital coordinates into ECEF coordinates.
            x = x_orb * cosO - y_orb * cosi * sinO  # ECEF X
            y = x_orb * sinO + y_orb * cosi * cosO  # ECEF Y
            z = y_orb * sini  # ECEF Z
            
            # Emit a tuple matching the downstream file format used by the script.
            yield (sod, 'G', prn, x, y, z, b, idx + 1, int(e['iode']),  # output record
                   e['sqrtA'], e['af0'], e['af1'], e['af2'],  # stored clock and orbit inputs
                   e['M0'], e['dn'], e['e'], e['omega'],  # mean anomaly, delta n, eccentricity, argument of perigee
                   e['cus'], e['cuc'], e['crs'], e['crc'], e['cis'], e['cic'],  # harmonic coefficients
                   e['i0'], e['idot'], e['Omega0'], e['Omegadot'])  # inclination and node parameters

## Example: Parse and Inspect

You can run the original functions to see them in action. Uncomment the cell below and adjust the path to a RINEX navigation file to experiment.

In [32]:
# Example usage (disabled by default):
ephs = parse_rinex_nav_v2('RINEX-NAV/brdc0060.25n')
per_sat = {prn: [] for prn in PRNS}
for eph in ephs:
    per_sat.setdefault(eph['prn'], []).append(eph)
rows = list(compute_positions_for_day(per_sat, doy=6, yy=25, step=900))
print(f"Parsed ephemerides: {len(ephs)}")
print(f"Sample output rows: {rows[:3]}")

Parsed ephemerides: 363
Sample output rows: [(0, 'G', 1, 16788003.684344176, 5176053.0645463485, 19929260.83028268, 7155.14768024476, 1, 104, 5153.7527504, 2.38670036197e-05, 3.3878677641e-11, 0.0, -3.07611675743, 4.51804533758e-09, 0.000224623014219, -1.22318081078, 9.34489071369e-06, 1.03563070297e-06, 19.46875, 197.75, 7.45058059692e-08, 3.72529029846e-08, 0.959724088452, -3.03584074067e-11, -1.90684230983, -8.10890919719e-09), (0, 'G', 2, 18424250.22231539, 6195476.084026008, 18738238.384947244, -82382.3383472246, 1, 9, 5153.68313026, -0.000274797901511, 9.09494701773e-12, 0.0, -3.08295736969, 4.30446501253e-09, 0.0165512511739, -1.06035770379, 9.46223735809e-06, 8.64267349243e-07, 14.6875, 195.46875, 3.33413481712e-07, 1.17346644402e-07, 0.966437928613, -9.96470078407e-11, -2.07157790572, -7.75532304033e-09), (0, 'G', 3, 18738305.39998167, -6335766.479130033, 17503117.333892643, 191963.14447051653, 1, 41, 5153.59826469, 0.000640320125967, 7.84439180279e-12, 0.0, -0.223430357651, 4